<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Sample notebook to build podsettings for gpu images from the lake user teamspace.

## Content
1. [Configuration](#Configuration)
2. [Build Podsetting](#Building the PodSetting for the Image)



### Configuration

In [ ]:
image_name = 'gpu-jupyter-user'
folder_name = 'aws-orbit_jupyter-user'

Get our orbit env and team names

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
user_name = %env USERNAME
namespace = %env AWS_ORBIT_USER_SPACE
account = %env ACCOUNT_ID
region=%env AWS_DEFAULT_REGION
(env_name,team_name, user_name, namespace, account, region)

Repository has an image (see below).  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [ ]:
image = f"{account}.dkr.ecr.{region}.amazonaws.com/orbit-regression/users/{image_name}"
image

### Building the PodSetting for the Image

In [ ]:
import json

customnameGPU = "orbit-gpu-image-sample-ps-"+team_name
description= " Machine Learning Image GPU - 2 CPU + 4G MEM + 1 GPU "
podsetting={
            "name": customnameGPU,
            "description": description,
            "image": image,
            "resources":{
                "limits":{            
                    "cpu": "2.0",
                    "memory": "4Gi",
                    "nvidia.com/gpu": "1"
                    },
                "requests":{
                    "cpu": "2.0",
                    "memory": "4Gi",
                    "nvidia.com/gpu": "1"
                    }
            },
            "node-group":"primary-gpu",
            "env":[
                {
                    "name":"source",
                    "value":"regressiontests"
                }
            ]
        }

with open("podsetting_data_gpu.json", 'w') as f:
    json.dump(podsetting, f)

### NOTE: "node-group":"primary-gpu" can be replaced with "instance-type":"g4dn.xlarge" 

In [ ]:
!orbit build podsetting --help 

Create the podsetting

In [ ]:
!orbit build podsetting --debug -e $env_name -t $team_name  podsetting_data.json

In [ ]:
import time 
time.sleep(3)

In [ ]:
!kubectl get podsettings -n$team_name|grep $customname

In [ ]:
!kubectl get poddefault -n$team_name|grep $customname

In [ ]:
!orbit delete podsetting --help

In [ ]:
!orbit delete podsetting -n $customname -t $team_name --debug

In [ ]:
import time 
time.sleep(3)

In [ ]:
!kubectl get podsettings -n$team_name|grep $customname

In [ ]:
!kubectl get poddefault -n$team_name|grep $customname